In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial.transform import Rotation as R
import random
import sys

def trajectory_distances(poses):
    distances = []
    distances.append(0)
    for i in range(1,len(poses)):
        p1 = poses[i-1]
        p2 = poses[i]
        delta = p1[0:3,3] - p2[0:3,3]
        distances.append(distances[i-1]+np.linalg.norm(delta))
    return distances

def last_frame_from_segment_length(dist,first_frame,length):
    for i in range(first_frame,len(dist)):
        if dist[i]>dist[first_frame]+length:
            return i
    return -1

def rotation_error(pose_error):
    a = pose_error[0,0]
    b = pose_error[1,1]
    c = pose_error[2,2]
    d = 0.5*(a+b+c-1)
    rot_error = np.arccos(max(min(d,1.0),-1.0))
    return rot_error

def translation_error(pose_error):
    dx = pose_error[0,3]
    dy = pose_error[1,3]
    dz = pose_error[2,3]
    return np.sqrt(dx*dx+dy*dy+dz*dz)

# def line2matrix(pose_line):
#     pose_line = np.array(pose_line)
#     pose_m = np.matrix(np.eye(4))
#     pose_m[0:3,:] = pose_line.reshape(3,4)
#     return pose_m

def calculate_sequence_error(poses_gt,poses_result,lengths=[10,20,30,40,50,60,70,80]):
    # error_vetor
    errors = []

    # paramet
    step_size = 1 #10; # every second
    num_lengths = len(lengths)

    # import ipdb;ipdb.set_trace()
    # pre-compute distances (from ground truth as reference)
    dist = trajectory_distances(poses_gt)
    # for all start positions do
    for  first_frame in range(0, len(poses_gt), step_size):
    # for all segment lengths do
        for i in range(0,num_lengths):
            #  current length
            length = lengths[i];

            # compute last frame
            last_frame = last_frame_from_segment_length(dist,first_frame,length);
            # continue, if sequence not long enough
            if (last_frame==-1):
                continue;

            # compute rotational and translational errors
            pose_delta_gt     = np.linalg.inv(poses_gt[first_frame]).dot(poses_gt[last_frame])
            pose_delta_result = np.linalg.inv(poses_result[first_frame]).dot(poses_result[last_frame])
            pose_error        = np.linalg.inv(pose_delta_result).dot(pose_delta_gt)
            r_err = rotation_error(pose_error);
            t_err = translation_error(pose_error);

            # compute speed
            num_frames = (float)(last_frame-first_frame+1);
            speed = length/(0.1*num_frames);

            # write to file
            error = [first_frame,r_err/length,t_err/length,length,speed]
            errors.append(error)
            # return error vector
    return errors

def calculate_ave_errors(errors,lengths=[10,20,30,40,50,60,70,80]):
    rot_errors=[]
    tra_errors=[]
    for length in lengths:
        rot_error_each_length =[]
        tra_error_each_length =[]
        for error in errors:
            if abs(error[3]-length)<0.1:
                rot_error_each_length.append(error[1])
                tra_error_each_length.append(error[2])

        if len(rot_error_each_length)==0:
            # import ipdb;ipdb.set_trace()
            continue
        else:
            rot_errors.append(sum(rot_error_each_length)/len(rot_error_each_length))
            tra_errors.append(sum(tra_error_each_length)/len(tra_error_each_length))
    return np.array(rot_errors)*180/np.pi, tra_errors

def eval(gt, data,kittitype=True):
    if kittitype:
        lens =  [100,200,300,400,500,600,700,800] #
    else:
        lens = [5,10,15,20,25,30,35,40] #[1,2,3,4,5,6] #
    errors = calculate_sequence_error(gt, data, lengths=lens)
    rot,tra = calculate_ave_errors(errors, lengths=lens)
    return np.mean(rot), np.mean(tra)

#kitti error
#----------------------------------------------
#transform stuff

def line2mat(line_data):
    mat = np.eye(4)
    mat[0:3,:] = line_data.reshape(3,4)
    return np.matrix(mat)

def motion2pose(data):
    data_size = len(data)
    all_pose = [] # np.zeros((data_size+1, 4, 4))
    all_pose.append(np.eye(4,4)) #[0,:] = np.eye(4,4)
    pose = np.eye(4,4)
    for i in range(0,data_size):
        pose = pose.dot(data[i])
        all_pose.append(pose)
    return all_pose

def pose2motion(data):
    data_size = len(data)
    all_motion = []
    for i in range(0,data_size-1):
        motion = np.linalg.inv(data[i]).dot(data[i+1])
        all_motion.append(motion)

    return np.array(all_motion) # N x 4 x 4

def SE2se(SE_data):
    result = np.zeros((6))
    result[0:3] = np.array(SE_data[0:3,3].T)
    result[3:6] = SO2so(SE_data[0:3,0:3]).T
    return result

def SO2so(SO_data):
    return R.from_dcm(SO_data).as_rotvec()

def so2SO(so_data):
    return R.from_rotvec(so_data).as_dcm()

def se2SE(se_data):
    result_mat = np.matrix(np.eye(4))
    result_mat[0:3,0:3] = so2SO(se_data[3:6])
    result_mat[0:3,3]   = np.matrix(se_data[0:3]).T
    return result_mat
### can get wrong result
def se_mean(se_datas):
    all_SE = np.matrix(np.eye(4))
    for i in range(se_datas.shape[0]):
        se = se_datas[i,:]
        SE = se2SE(se)
        all_SE = all_SE*SE
    all_se = SE2se(all_SE)
    mean_se = all_se/se_datas.shape[0]
    return mean_se

def ses_mean(se_datas):
    se_datas = np.array(se_datas)
    se_datas = np.transpose(se_datas.reshape(se_datas.shape[0],se_datas.shape[1],se_datas.shape[2]*se_datas.shape[3]),(0,2,1))
    se_result = np.zeros((se_datas.shape[0],se_datas.shape[2]))
    for i in range(0,se_datas.shape[0]):
        mean_se = se_mean(se_datas[i,:,:])
        se_result[i,:] = mean_se
    return se_result

def ses2poses(data):
    data_size = data.shape[0]
    all_pose = np.zeros((data_size+1,12))
    temp = np.eye(4,4).reshape(1,16)
    all_pose[0,:] = temp[0,0:12]
    pose = np.matrix(np.eye(4,4))
    for i in range(0,data_size):
        data_mat = se2SE(data[i,:])
        pose = pose*data_mat
        pose_line = np.array(pose[0:3,:]).reshape(1,12)
        all_pose[i+1,:] = pose_line
    return all_pose

def SEs2ses(motion_data):
    data_size = motion_data.shape[0]
    ses = np.zeros((data_size,6))
    for i in range(0,data_size):
        SE = np.matrix(np.eye(4))
        SE[0:3,:] = motion_data[i,:].reshape(3,4)
        ses[i,:] = SE2se(SE)
    return ses

def so2quat(so_data):
    so_data = np.array(so_data)
    theta = np.sqrt(np.sum(so_data*so_data))
    axis = so_data/theta
    quat=np.zeros(4)
    quat[0:3] = np.sin(theta/2)*axis
    quat[3] = np.cos(theta/2)
    return quat

def quat2so(quat_data):
    quat_data = np.array(quat_data)
    sin_half_theta = np.sqrt(np.sum(quat_data[0:3]*quat_data[0:3]))
    axis = quat_data[0:3]/sin_half_theta
    cos_half_theta = quat_data[3]
    theta = 2*np.arctan2(sin_half_theta,cos_half_theta)
    so = theta*axis
    return so

# input so_datas batch*channel*height*width
# return quat_datas batch*numner*channel
def sos2quats(so_datas,mean_std=[[1],[1]]):
    so_datas = np.array(so_datas)
    so_datas = so_datas.reshape(so_datas.shape[0],so_datas.shape[1],so_datas.shape[2]*so_datas.shape[3])
    so_datas = np.transpose(so_datas,(0,2,1))
    quat_datas = np.zeros((so_datas.shape[0],so_datas.shape[1],4))
    for i_b in range(0,so_datas.shape[0]):
        for i_p in range(0,so_datas.shape[1]):
            so_data = so_datas[i_b,i_p,:]
            quat_data = so2quat(so_data)
            quat_datas[i_b,i_p,:] = quat_data
    return quat_datas

def SO2quat(SO_data):
    rr = R.from_dcm(SO_data)
    return rr.as_quat()

def quat2SO(quat_data):
    return R.from_quat(quat_data).as_dcm()


def pos_quat2SE(quat_data):
    SO = R.from_quat(quat_data[3:7]).as_dcm()
    SE = np.matrix(np.eye(4))
    SE[0:3,0:3] = np.matrix(SO)
    SE[0:3,3]   = np.matrix(quat_data[0:3]).T
    SE = np.array(SE[0:3,:]).reshape(1,12)
    return SE


def pos_quats2SEs(quat_datas):
    data_len = quat_datas.shape[0]
    SEs = np.zeros((data_len,12))
    for i_data in range(0,data_len):
        SE = pos_quat2SE(quat_datas[i_data,:])
        SEs[i_data,:] = SE
    return SEs


def pos_quats2SE_matrices(quat_datas):
    data_len = quat_datas.shape[0]
    SEs = []
    for quat in quat_datas:
        SO = R.from_quat(quat[3:7]).as_dcm()
        SE = np.eye(4)
        SE[0:3,0:3] = SO
        SE[0:3,3]   = quat[0:3]
        SEs.append(SE)
    return SEs

def SE2pos_quat(SE_data):
    pos_quat = np.zeros(7)
    pos_quat[3:] = SO2quat(SE_data[0:3,0:3])
    pos_quat[:3] = SE_data[0:3,3].T
    return pos_quat

def shift0(traj):
    '''
    Traj: a list of [t + quat]
    Return: translate and rotate the traj
    '''
    traj_ses = pos_quats2SE_matrices(np.array(traj))
    traj_init = traj_ses[0]
    traj_init_inv = np.linalg.inv(traj_init)
    new_traj = []
    for tt in traj_ses:
        ttt=traj_init_inv.dot(tt)
        new_traj.append(SE2pos_quat(ttt))
    return np.array(new_traj)

def ned2cam(traj):
    '''
    transfer a ned traj to camera frame traj
    '''
    T = np.array([[0,1,0,0],
                  [0,0,1,0],
                  [1,0,0,0],
                  [0,0,0,1]], dtype=np.float32)
    T_inv = np.linalg.inv(T)
    new_traj = []
    traj_ses = tf.pos_quats2SE_matrices(np.array(traj))

    for tt in traj_ses:
        ttt=T.dot(tt).dot(T_inv)
        new_traj.append(tf.SE2pos_quat(ttt))

    return np.array(new_traj)

def cam2ned(traj):
    '''
    transfer a camera traj to ned frame traj
    '''
    T = np.array([[0,0,1,0],
                  [1,0,0,0],
                  [0,1,0,0],
                  [0,0,0,1]], dtype=np.float32)
    T_inv = np.linalg.inv(T)
    new_traj = []
    traj_ses = tf.pos_quats2SE_matrices(np.array(traj))

    for tt in traj_ses:
        ttt=T.dot(tt).dot(T_inv)
        new_traj.append(tf.SE2pos_quat(ttt))

    return np.array(new_traj)


def trajectory_transform(gt_traj, est_traj):
    '''
    1. center the start frame to the axis origin
    2. align the GT frame (NED) with estimation frame (camera)
    '''
    gt_traj_trans = shift0(gt_traj)
    est_traj_trans = shift0(est_traj)

    # gt_traj_trans = ned2cam(gt_traj_trans)
    # est_traj_trans = cam2ned(est_traj_trans)

    return gt_traj_trans, est_traj_trans

def rescale_bk(poses_gt, poses):
    motion_gt = tf.pose2motion(poses_gt)
    motion    = tf.pose2motion(poses)

    speed_square_gt = np.sum(motion_gt[:,0:3,3]*motion_gt[:,0:3,3],1)
    speed_gt = np.sqrt(speed_square_gt)
    speed_square    = np.sum(motion[:,0:3,3]*motion[:,0:3,3],1)
    speed = np.sqrt(speed_square)
    # when the speed is small, the scale could become very large
    # import ipdb;ipdb.set_trace()
    mask = (speed_gt>0.0001) # * (speed>0.00001)
    scale = np.mean((speed[mask])/speed_gt[mask])
    scale = 1.0/scale
    motion[:,0:3,3] = motion[:,0:3,3]*scale
    pose_update = tf.motion2pose(motion)
    return  pose_update, scale

def pose2trans(pose_data):
    data_size = len(pose_data)
    trans = []
    for i in range(0,data_size-1):
        tran = np.array(pose_data[i+1][:3]) - np.array(pose_data[i][:3]) # np.linalg.inv(data[i]).dot(data[i+1])
        trans.append(tran)

    return np.array(trans) # N x 3


def rescale(poses_gt, poses):
    '''
    similar to rescale
    poses_gt/poses: N x 7 poselist in quaternion format
    '''
    trans_gt = pose2trans(poses_gt)
    trans    = pose2trans(poses)

    speed_square_gt = np.sum(trans_gt*trans_gt,1)
    speed_gt = np.sqrt(speed_square_gt)
    speed_square    = np.sum(trans*trans,1)
    speed = np.sqrt(speed_square)
    # when the speed is small, the scale could become very large
    # import ipdb;ipdb.set_trace()
    mask = (speed_gt>0.0001) # * (speed>0.00001)
    scale = np.mean((speed[mask])/speed_gt[mask])
    scale = 1.0/scale
    poses[:,0:3] = poses[:,0:3]*scale
    return  poses, scale

def trajectory_scale(traj, scale):
    for ttt in traj:
        ttt[0:3,3] = ttt[0:3,3]*scale
    return traj

def timestamp_associate(first_list, second_list, max_difference):
    """
    Associate two trajectory of [stamp,data]. As the time stamps never match exactly, we aim
    to find the closest match for every input tuple.

    Input:
    first_list -- first list of (stamp,data)
    second_list -- second list of (stamp,data)
    max_difference -- search radius for candidate generation

    Output:
    first_res: matched data from the first list
    second_res: matched data from the second list

    """
    first_dict = dict([(l[0],l[1:]) for l in first_list if len(l)>1])
    second_dict = dict([(l[0],l[1:]) for l in second_list if len(l)>1])

    first_keys = first_dict.keys()
    second_keys = second_dict.keys()
    potential_matches = [(abs(a - b ), a, b)
                         for a in first_keys
                         for b in second_keys
                         if abs(a - b) < max_difference]
    potential_matches.sort()
    matches = []
    for diff, a, b in potential_matches:
        if a in first_keys and b in second_keys:
            first_keys.remove(a)
            second_keys.remove(b)
            matches.append((a, b))

    matches.sort()

    first_res = []
    second_res = []
    for t1, t2 in matches:
        first_res.append(first_dict[t1])
        second_res.append(second_dict[t2])
    return np.array(first_res), np.array(second_res)

def transform_trajs(gt_traj, est_traj, cal_scale):
    gt_traj, est_traj = trajectory_transform(gt_traj, est_traj)
    if cal_scale :
        est_traj, s = rescale(gt_traj, est_traj)
        print('  Scale, {}'.format(s))
    else:
        s = 1.0
    return gt_traj, est_traj, s

#ate
def evaluate_ate(gt_traj, est_traj, scale):
    gt_xyz = np.matrix(gt_traj[:, 0:3].transpose())
    est_xyz = np.matrix(est_traj[:, 0:3].transpose())

    rot, trans, trans_error, s = align(gt_xyz, est_xyz, scale)
    print('  ATE scale: {}'.format(s))
    error = np.sqrt(np.dot(trans_error, trans_error) / len(trans_error))

    # align two trajs
    est_SEs = pos_quats2SE_matrices(est_traj)
    T = np.eye(4)
    T[:3, :3] = rot
    T[:3, 3:] = trans
    T = np.linalg.inv(T)
    est_traj_aligned = []
    for se in est_SEs:
        se[:3, 3] = se[:3, 3] * s
        se_new = T.dot(se)
        se_new = SE2pos_quat(se_new)
        est_traj_aligned.append(se_new)

    est_traj_aligned = np.array(est_traj_aligned)
    return error, gt_traj, est_traj_aligned

def quats2SEs(gt_traj, est_traj):
    gt_SEs = pos_quats2SE_matrices(gt_traj)
    est_SEs = pos_quats2SE_matrices(est_traj)
    return gt_SEs, est_SEs

def pos_quats2SE_matrices(quat_datas):
    data_len = quat_datas.shape[0]
    SEs = []
    for quat in quat_datas:
        SO = R.from_quat(quat[3:7]).as_matrix()
        SE = np.eye(4)
        SE[0:3,0:3] = SO
        SE[0:3,3]   = quat[0:3]
        SEs.append(SE)
    return SEs

#---------------------------------------------------------------------
#rpe
def ominus(a,b):
    """
    Compute the relative 3D transformation between a and b.

    Input:
    a -- first pose (homogeneous 4x4 matrix)
    b -- second pose (homogeneous 4x4 matrix)

    Output:
    Relative 3D transformation from a to b.
    """
    return np.dot(np.linalg.inv(a),b)

def compute_distance(transform):
    """
    Compute the distance of the translational component of a 4x4 homogeneous matrix.
    """
    return np.linalg.norm(transform[0:3,3])

def compute_angle(transform):
    """
    Compute the rotation angle from a 4x4 homogeneous matrix.
    """
    # an invitation to 3-d vision, p 27
    return np.arccos( min(1,max(-1, (np.trace(transform[0:3,0:3]) - 1)/2) ))

def distances_along_trajectory(traj):
    """
    Compute the translational distances along a trajectory.
    """
    motion = [ominus(traj[i+1],traj[i]) for i in range(len(traj)-1)]
    distances = [0]
    sum = 0
    for t in motion:
        sum += compute_distance(t)
        distances.append(sum)
    return distances


def evaluate_trajectory(traj_gt, traj_est, param_max_pairs=10000, param_fixed_delta=False,
                        param_delta=1.00):
    """
    Compute the relative pose error between two trajectories.

    Input:
    traj_gt -- the first trajectory (ground truth)
    traj_est -- the second trajectory (estimated trajectory)
    param_max_pairs -- number of relative poses to be evaluated
    param_fixed_delta -- false: evaluate over all possible pairs
                         true: only evaluate over pairs with a given distance (delta)
    param_delta -- distance between the evaluated pairs
    param_delta_unit -- unit for comparison:
                        "s": seconds
                        "m": meters
                        "rad": radians
                        "deg": degrees
                        "f": frames
    param_offset -- time offset between two trajectories (to model the delay)
    param_scale -- scale to be applied to the second trajectory

    Output:
    list of compared poses and the resulting translation and rotation error
    """

    if not param_fixed_delta:
        if(param_max_pairs==0 or len(traj_est)<np.sqrt(param_max_pairs)):
            pairs = [(i,j) for i in range(len(traj_est)) for j in range(len(traj_est))]
        else:
            pairs = [(random.randint(0,len(traj_est)-1),random.randint(0,len(traj_est)-1)) for i in range(param_max_pairs)]
    else:
        pairs = []
        for i in range(len(traj_est)):
            j = i + param_delta
            if j < len(traj_est):
                pairs.append((i,j))
        if(param_max_pairs!=0 and len(pairs)>param_max_pairs):
            pairs = random.sample(pairs,param_max_pairs)

    result = []
    for i,j in pairs:

        error44 = ominus(  ominus( traj_est[j], traj_est[i] ),
                           ominus( traj_gt[j], traj_gt[i] ) )

        trans = compute_distance(error44)
        rot = compute_angle(error44)

        result.append([i,j,trans,rot])

    if len(result)<2:
        raise Exception("Couldn't find pairs between groundtruth and estimated trajectory!")

    return result

#--------------------------------------------------------------------
#scale
def align(model,data,calc_scale=True):
    """Align two trajectories using the method of Horn (closed-form).

    Input:
    model -- first trajectory (3xn)
    data -- second trajectory (3xn)

    Output:
    rot -- rotation matrix (3x3)
    trans -- translation vector (3x1)
    trans_error -- translational error per point (1xn)

    """
    np.set_printoptions(precision=3,suppress=True)
    print(model.mean(1))
    #mean_values = np.mean(model, axis=0)
    #model_zerocentered = model - mean_values
    model_zerocentered = model - model.mean(1)
    data_zerocentered = data - data.mean(1)

    W = np.zeros( (3,3) )
    for column in range(model.shape[1]):
        #print(model_zerocentered[:,column].shape, data_zerocentered.shape)
        W += np.outer(model_zerocentered[:,column],data_zerocentered[:,column])
        #W += np.outer(model_zerocentered[:,column],data_zerocentered[:,column:column+1])
    U,d,Vh = np.linalg.linalg.svd(W.transpose())
    S = np.matrix(np.identity( 3 ))
    if(np.linalg.det(U) * np.linalg.det(Vh)<0):
        S[2,2] = -1
    rot = U*S*Vh

    if calc_scale:
        rotmodel = rot*model_zerocentered
        dots = 0.0
        norms = 0.0
        for column in range(data_zerocentered.shape[1]):
            dots += np.dot(data_zerocentered[:,column].transpose(),rotmodel[:,column])
            normi = np.linalg.norm(model_zerocentered[:,column])
            norms += normi*normi
        # s = float(dots/norms)
        s = float(norms/dots)
    else:
        s = 1.0

    # trans = data.mean(1) - s*rot * model.mean(1)
    # model_aligned = s*rot * model + trans
    # alignment_error = model_aligned - data

    # scale the est to the gt, otherwise the ATE could be very small if the est scale is small
    trans = s*data.mean(1) - rot * model.mean(1)
    model_aligned = rot * model + trans
    data_alingned = s * data
    alignment_error = model_aligned - data_alingned

    trans_error = np.sqrt(np.sum(np.multiply(alignment_error,alignment_error),0)).A[0]

    return rot,trans,trans_error, s

def plot_traj(ax,stamps,traj,style,color,label):
    """
    Plot a trajectory using matplotlib.

    Input:
    ax -- the plot
    stamps -- time stamps (1xn)
    traj -- trajectory (3xn)
    style -- line style
    color -- line color
    label -- plot legend

    """
    stamps.sort()
    interval = np.median([s-t for s,t in zip(stamps[1:],stamps[:-1])])
    x = []
    y = []
    last = stamps[0]
    for i in range(len(stamps)):
        if stamps[i]-last < 2*interval:
            x.append(traj[i][0])
            y.append(traj[i][1])
        elif len(x)>0:
            ax.plot(x,y,style,color=color,label=label)
            label=""
            x=[]
            y=[]
        last= stamps[i]
    if len(x)>0:
        ax.plot(x,y,style,color=color,label=label)

#--------------------------------------------------------

def SO2quat(SO_data):
    rr = R.from_matrix(SO_data)
    return rr.as_quat()

def SE2pos_quat(SE_data):
    pos_quat = np.zeros(7)
    pos_quat[3:] = SO2quat(SE_data[0:3,0:3])
    pos_quat[:3] = SE_data[0:3,3].T
    return pos_quat

def kitti2tartan(traj):
    '''
    traj: in kitti style, N x 12 numpy array, in camera frame
    output: in TartanAir style, N x 7 numpy array, in NED frame
    '''
    T = np.array([[0,0,1,0],
                  [1,0,0,0],
                  [0,1,0,0],
                  [0,0,0,1]], dtype=np.float32)
    T_inv = np.linalg.inv(T)
    new_traj = []

    for pose in traj:
        tt = np.eye(4)
        tt[:3,:] = pose.reshape(3,4)
        ttt=T.dot(tt).dot(T_inv)
        new_traj.append(SE2pos_quat(ttt))

    return np.array(new_traj)
#--------------------------------------------------------

# ESTIMATED TRAJECTORY
file_id = "1gdDdMKeDk0eH3DoCHr4-CPzUGBSCircH"
!gdown https://drive.google.com/uc?id={file_id}
est = np.loadtxt("V_203.txt")

# GROUND TRUTH
file_id = "17tlRwcECjVApKoNmbN_p7Rswx5HtHPGM"
!gdown https://drive.google.com/uc?id={file_id}

dataset = 0
if dataset == 1: # KITTI to Tartan
    gt = np.loadtxt("10.txt")
    convert = kitti2tartan(gt)
else: # EUROC to Tartan
    #from google.colab import drive
    #drive.mount('/content/gdrive/')
    gt = pd.read_csv("V2_03_difficult.csv")
    splice = gt.iloc[1:, 1:8] # remove timestamp (col 0), col 1-8 = xyz + quaternion)
    scale = splice[::12]
    convert = scale.iloc[:-25]
    convert = np.array(convert) # result matches shape of est

np.savetxt("converted.txt", convert)

#cut_gt = convert[:, :3]
#swapped_gt = cut_gt.T
#cut_est = est[:, :3]
#swapped_est = cut_est.T

# transform and scale
gt_traj_trans, est_traj_trans, s = transform_trajs(convert, est, True)
gt_SEs, est_SEs = quats2SEs(gt_traj_trans, est_traj_trans)

# ATE
ate_score, gt_ate_aligned, est_ate_aligned = evaluate_ate(convert, est, True)
print(ate_score)
f = open('ate_result.txt', 'w')
print(ate_score, file=f)
np.savetxt("gt_aligned.txt", gt_ate_aligned)
np.savetxt("est_aligned.txt", est_ate_aligned)

# RPE
rpe_result = evaluate_trajectory(gt_SEs, est_SEs, param_max_pairs=10000, param_fixed_delta=False,
                        param_delta=1.00)
np.savetxt("rpe.txt", rpe_result)
arr = np.array(rpe_result)
#actual = arr[:, 0]
#predicted = arr[:, 1]
#squared_diff = (actual - predicted) ** 2
#mse = np.mean(squared_diff)
#rmse = np.sqrt(mse)
trell = np.mean(arr[:, 2])
rrell = np.mean(arr[:, 3])
print(trell, rrell)

# KITTI
if dataset == 1: # if KITTI
    kitti = eval(gt_SEs, est_SEs,kittitype=True)
    np.savetxt("kitti.txt", kitti)

Downloading...
From: https://drive.google.com/uc?id=1gdDdMKeDk0eH3DoCHr4-CPzUGBSCircH
To: /content/V_203.txt
100% 336k/336k [00:00<00:00, 4.43MB/s]
Downloading...
From: https://drive.google.com/uc?id=17tlRwcECjVApKoNmbN_p7Rswx5HtHPGM
To: /content/V2_03_difficult.csv
100% 3.92M/3.92M [00:00<00:00, 37.1MB/s]
  Scale, 0.5813974239161215
[[-0.67 ]
 [ 0.464]
 [ 1.838]]
  ATE scale: 2.91429869949148
11.58401727587788
3.9555409385175904 1.9623169352626424
